<h1 align="center"> Las raíces Lispianas de Julia </h1>
<h2 align="center"> Workshop Juliero 2024 </h1>
<h3 align="center"> Juan I. Perotti </h2>
<h3 align="center"> 2024-07-29 </h2>

### Abstracciones

Porqué usamos lenguajes de programación en vez de programar en código máquina, o quizás en assembler? Porque habilitan el uso de abstracciones que nos facilitan razonar sobre nuestros programas.

### Antiguamente...

Antiguamente la gente programaba en pseudocódigo en papel y luego "traducía" dicho pseudocódigo a instrucciones en **código máquina**, o quizás a **código assembler**. 
Esta traducción es lo que llamamos "compilar un programa". Compilar de código assembler a codigo máquina es trivial, ya que la traducción es casi literal. Por otro lado, compilar de pseudo-código puede ser muy complicado, ya que generalmente no existe una única forma de hacerlo.

### John Backus

En 1957 un equipo liderado por John Backus introduce FORTRAN I (FORmula TRANslator), el primer lenguage de programación que viene acompañado de un programa que automatiza la compilación de código FORTRAN a código máquina, i.e., lo que comunmente llamamos un compilador.

### Funciones

Las funciones están entre las abstracciones más utilizadas por un lenguaje de programación. 
Sin embargo, es importante notar que la noción de función que encontramos en computación es distinta a la que encontramos en matemáticas. Las funciones matemáticas son inmutables, mientras que en computación las funciones pueden afectar y ser afectadas por variables externas.

In [ ]:
# Ejemplo

a = 0

function f()
   global a += 1
end

function g()
   return a+1 
end

In [ ]:
g()

In [ ]:
f()

In [ ]:
g()

### Lenguajes procedurales

Como se implementa el llamado a función en lenguajes procedurales como FORTRAN o C? 

Cuando una función llama a otra, la secuencia de instrucciones que ejecuta la computadora sufre un salto hacia un fragmento de código que computa la función.

Al culminar el cómputo de la función, el flujo del programa debe retornar al punto de llamada, por lo que la dirección de llamada debe ser almacenada **temporalmente** en alguna parte.

Las direcciones de retorno tienen que ser almacenadas en variables temporales porque el flujo del programa es generalmente impredecible en tiempos de compilación.
Esto resulta particularmente importante si se pretende usar **recursión**.

    Pseudo-código                      Assembler
    -------------                      ---------
    
    function f(...)                    :f
        ...                            ...
        g(...)                         call g
        ...                            ...
        return ...                     ret
    end
    
    function g(...)                    :g
        ...                            ...
        h()                            call h
        ...                            ...
        return ...                     ret
    end
    
    function h(...)                    :h
        ...                            ...
        if ...                         jz end_1
            g()                        call g
        end                            :end_1
        ...                            ...
        return ...                     ret
    end

### Stack

Lenguajes como FORTRAN o C almacenan las direcciones de retorno en una **pila** o **stack**, ya que los retornos ocurren en el orden inverso a los llamados. 
Además, en la pila se pueden instanciar las variables temporales que contienen los argumentos de los llamados a funciones.

Usar una pila para instanciar llamados a funciones presenta varias ventajas. 
Es fácilmente implementable y optimizable (ej. inlining). 
Sin embargo, también fuerza ciertas limitaciones.

<img src="images/stack.png" width="300"/>

### Lenguajes funcionales



Muchos de Ud. seguramente conocen funcionales en matemática. Son funciones que "comen" otras funciones como argumentos, en vez de números o vectores.

Un ejemplo es el la acción.
Otro es operador derivada $D$, que come una función $f$ y devuelve su derivada $f'$.

Las funcionales también son muy útiles en computación. 
Sin embargo, los lenguajes como FORTRAN o C no permiten la programación funcional. 
La razón de porqué es así tiene nombre.

### The FUNARG problem

Consideren el siguiente pseudo-código.

    function f(x)
        function g(y)
            x += 1
            return x+y
        end
        return g
    end
    
Aquí, una llamada a `f` crea y retorna una función `g`, que a su vez depende del parámetro `x` de `f`. La función `g` crea un **closure** sobre la instancia de la variable `x` creada por la llamada a `f`.

Cada vez que se llama a `f` se crea una instancia de `x` y una instancia de `g`. Así, por ejemplo, se tiene que

    g1 = f(0)
    g2 = f(100)
    
    print g1(10)  # g1 setea su instancia de x a 1 e imprime 1+10=11
    print g2(10)  # g2 setea su instancia de x a 101 e imprime 101+10=111
    print g1(10)  # g1 setea su instancia de x a 2 e imprime 2+10=12
    
Si este pseudo-código instanciase las variables de llamados a función como lo hacen FORTRAN o C, luego las instancias de `x` se crearían en el **stack** durante las correspondientes llamadas a `f`, y luego estas serían destruídas cuando las llamadas a `f` retornen. 
En consecuencia, estas instancias dejarían de existir para que puedan ser usadas por las instancias creadas de `g`.

Este es un ejemplo del **FUNARG problem**. En los lenguajes funcionales las variables temporales no pueden instanciarse en el *stack*.

In [ ]:
function f(x)
    function g(y)
        x += 1
        return x+y
    end
    return g
end

g1 = f(0)
g2 = f(100)
@show g1(10)
@show g2(10)
@show g1(10)
;

### Garbage collection

Debido al FUNARG problem, los lenguajes funcionales crean instancias de variables temporales en un **heap**.

Un heap es lo que permite reservar y liberar memoria en tiempo de ejecución, tal como se ocurre cuando invocamos `allocate` y `deallocate` en FORTRAN o `malloc` y `free` en C.

Además de un heap, los lenguajes funcionales utilizan un **garbage collector** para liberar automaticamente la memoria utilizada por las variables temporales cuando dejan de ser usadas.

Si bien han mejorado mucho, los garbage collectors suelen ser computacionalmente costosos en comparación al uso de un stack, por lo que conviene evitar el uso de "funcionales" en las partes del código críticas para la velocidad.

### Metaprogramming

La **metaprogramación** consiste en escribir programas que "modifiquen" o "escriban" programas. 
En algún sentido, podemos pensar a la programación funcional como una especie de metaprogramación.

Julia es un lenguaje que soporta el paradigma de metaprogramación:

* [https://docs.julialang.org/en/v1/manual/metaprogramming/](https://docs.julialang.org/en/v1/manual/metaprogramming/)

Nosotros inspeccionaremos algunos de los conceptos esenciales en LISP.

### LISP (LISt Processing)

En 1958-1959 John McCarthy introduce **LISP**, un lenguaje pionero en la metaprogramación, la programación funcional y el uso de un garbage collector.

#### Polish notation

La sintaxis de LISP es rara. Está inspirada en la **notación Polaca**. Veamos de que trata.

Normalmente denotamos el llamado a una función usando

    f(a,b)
    
En notación polaca esto toma la forma

    (f a b)
    
Es decir, se usa una tupla o **lista** en donde el primer elemento es la función, operador o instrucción y los elementos restantes son los argumentos de la expressión.

De manera similar, denotamos una expressión condicional por

    if A
        B
    else
        C
    end
    
donde, `A`, `B` y `C` son expresiones arbitrarias. En notación polaca esto se denota por

    (if A B C)
    
Notar que aquí `if` no es un llamado a función, ya que los argumentos `B` y `C` se evalúan o no dependiendo del resultado de la evaluación de `A`, mientras que en un llamado a función todos los argumentos se evalúan.

#### S-expressions

Las expresiones en LISP se llaman **S-expressions** y están conformadas por

1. Constantes. Ej. números `1`, `3.14`, ...
2. Símbolos. Ej. `x`, `if`, `+`, ...
3. Listas de otras expresiones. Ej. `(if x (+ y z) 0)`

Es importante resaltar que los **símbolos** son distintos de las **variables**. Los símbolos son más generales, pueden usarse como variables pero no al revés.

#### Quotation

Además de instrucciones que permiten manipular expresiones, incluyendo listas, LISP permite **"citar"** expresiones usando la instrucción `quote`. De modo que

    (quote A)
    
evalúa a la expresión

    A
    
Esto se usa tanto que LISP incluye *syntactic sugar* para el uso de quotes. Específicamente `(quote A)` es equivalente a escribir `'A`.

#### Homocoinicity

El uso de **S-expressions** y **quotations** permite usar una misma sintaxis para representar **código** y **datos**. 
Esta propiedad se llama **homocoinicidad** y facilita significativamente la metaprogramación.

### Cálculos simbólicos

La metaprogramación facilita enormemente el cálculo simbólico y el desarrollo de compiladores.

Existen varios programas/sistemas desarrollados para el cálculo simbólico que usan o están inspirados (al menos en parte) en LISP. 
Por ejemplo:

    https://maxima.sourceforge.io/

Wolfram Matemática toma algunas de estas ideas.
 
### Compiladores 

La metaprogramación es particularmente útil para el desarrollo de compiladores. 
De hecho, el uso de Abstract Syntax Tress (AST) (un pariente de las S-expressions) es común en el desarrollo de compiladores.

### Bootstrapping

**Bootstrapping** es el arte de desarrollar un compilador para un lenguaje **X** en **X** mismo. 
Se comienza escribiendo un pequeño interprete o compilador de una versión reducida **X'** de **X** en algún otro lenguaje (típicamente C) y luego se usa **X'** para escribir un mejor interprete/compilador **X''** de **X** y así hasta que se converge a **X**.

La metaprogramación facilita el bootstrappeado de lenguajes funcionales. 

### Las raíces LISPianas de Julia
 
Julia tiene escondido un interprete de LISP llamado `femptolisp`. 
Femptolisp tuvo un rol importante en el desarrollo temprano de Julia. 
En particular, con femptolisp se desarrolló el primer parser de código Julia y luego se lo utilizó para "bootstrappear" Julia. También se lo utilizó para experimentar con diferentes aspectos de la sintaxis y la semántica de Julia.

Se lo puede usar escribiendo:

    juan@laptop:workshop-juliero-2024$ julia --lisp
    ;  _
    ; |_ _ _ |_ _ |  . _ _
    ; | (-||||_(_)|__|_)|_)
    ;-------------------|----------------------------------------------------------

    > (+ 1 2)
    3

    > ((lambda (x y) (+ x y)) 1 2)
    3

    > (exit)
    juan@laptop:workshop-juliero-2024$